Illia Bannitsyn illiabann@student.agh.edu.pl

Patrycja Marchwica patmar@student.agh.edu.pl

# Import danych

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from keras import models
from keras import layers
from keras.utils import to_categorical
from keras.wrappers.scikit_learn import KerasClassifier
import tensorflow as tf
from tensorflow.compat.v1.keras import backend as K


#config = tf.compat.v1.ConfigProto( device_count = {'GPU': 1 , 'CPU': 23} , log_device_placement=True )
#sess = tf.compat.v1.Session(config=config) 
#K.set_session(sess)
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()


ModuleNotFoundError: No module named 'matplotlib'

# Funkcje

## Deskew



In [ ]:
SZ=28
affine_flags = cv2.WARP_INVERSE_MAP|cv2.INTER_LINEAR

def deskew(img):
    m = cv2.moments(img)
    if abs(m['mu02']) < 1e-2:
        return img.copy()
    skew = m['mu11']/m['mu02']
    M = np.float32([[1, skew, -0.5*SZ*skew], [0, 1, 0]])
    img = cv2.warpAffine(img,M,(SZ, SZ),flags=affine_flags)
    return img

## HOG (Histogram of Oriented Gradients)



In [ ]:
def HOG(im):
    winSize = (20,20)
    blockSize = (10,10)
    blockStride = (5,5)
    cellSize = (10,10)
    nbins = 9
    derivAperture = 1
    winSigma = -1.
    histogramNormType = 0
    L2HysThreshold = 0.2
    gammaCorrection = 1
    nlevels = 64
    signedGradients = True

    hog = cv2.HOGDescriptor(winSize,blockSize,blockStride,cellSize,nbins,derivAperture,winSigma,histogramNormType,L2HysThreshold,gammaCorrection,nlevels, signedGradients)
    descriptor = np.ravel(hog.compute(im))
    
    return descriptor

## Tworzenie modelu dla sieci


In [ ]:
def create_model():
  model = models.Sequential()
  model.add(layers.Dense(512, activation='relu'))
  model.add(layers.Dense(10, activation='softmax'))
  model.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])
  return model

## Grid Search - Walidacja krzyżowa

In [ ]:
def SVMGridSearch(train_data, train_labels):
  parameters = {'kernel':('linear', 'rbf'), 'C': np.linspace(start = 0.001, stop = 2, num = 50)}
  svc = svm.SVC()
  clf = GridSearchCV(svc, parameters, scoring='accuracy', cv=StratifiedKFold())
  clf.fit(train_data, train_labels)
  best_params = clf.best_params_
  my_svc = clf.best_estimator_
  
  print("Best hiperparameters: ", best_params)
  
  return my_svc

In [ ]:
def RandomForestGridSearch(train_data, train_labels):
  parameters = {
    'max_depth': [5, 8, 15, 25, 30],
    'min_samples_leaf': [1, 2, 5, 10],
    'min_samples_split': [2, 5, 10, 15, 100],
    'n_estimators': [20, 30, 50, 70, 100]
  }

  rf = RandomForestClassifier()
  clf = GridSearchCV(estimator = rf, param_grid = parameters, cv = 3, n_jobs = -1, verbose = 1)
  clf.fit(train_data, train_labels)
  best_params = clf.best_params_
  my_rf = clf.best_estimator_
  
  print("Best hiperparameters: ", best_params)

  return my_rf

In [ ]:
def NeuralNetworkGridSrearch(train_data, train_labels):
  parameters = {
      'batch_size': [30, 50, 100],
      'epochs': [5, 10, 15]
      }
  kc = KerasClassifier(build_fn=create_model, verbose=0)
  clf = GridSearchCV(estimator=kc, param_grid=parameters, n_jobs=-1, cv=3)
  clf.fit(train_data, train_labels)
  best_params = clf.best_params_

  print("Best hiperparameters: ", best_params)

  return best_params

## Macierz pomyłek i raporty

In [ ]:
def getConfMatrix(test_labels, pred_labels_deskewed, pred_labels_non_deskewed,pred_labels_raw_deskewed,pred_labels_shuffle):
  cm_deskewed = confusion_matrix(test_labels, pred_labels_deskewed)
  cm_non_deskewed = confusion_matrix(test_labels, pred_labels_non_deskewed)
  cm_raw_deskewed = confusion_matrix(test_labels, pred_labels_raw_deskewed)
  cm_shuffle = confusion_matrix(test_labels, pred_labels_shuffle)


  print("Macierz pomyłek - dane wyrównane:")
  print(cm_deskewed)
  print("Macierz pomyłek - dane niewyrównane:")
  print(cm_non_deskewed)
  print("Macierz pomyłek - dane surowe:")
  print(cm_raw_deskewed)
  print("Macierz pomyłek - dane zmieszane:")
  print(cm_shuffle)

In [ ]:
def getReports(test_labels, pred_labels_deskewed, pred_labels_non_deskewed,pred_labels_raw_deskewed,pred_labels_shuffle):
  print("Raport - dane wyrównane:")
  print(classification_report(test_labels, pred_labels_deskewed))
  print("Raport - dane niewyrównane:")
  print(classification_report(test_labels, pred_labels_non_deskewed))
  print("Raport - dane surowe:")
  print(classification_report(test_labels, pred_labels_raw_deskewed))
  print("Raport - dane zmieszane:")
  print(classification_report(test_labels, pred_labels_shuffle))

In [ ]:
def getAccuracy(test_labels, pred_labels_deskewed, pred_labels_non_deskewed,pred_labels_raw_deskewed,pred_labels_shuffle):
  print("Accuracy - dane wyrównane: " + str(accuracy_score(test_labels, pred_labels_deskewed)))
  print("Accuracy - dane niewyrównane: " + str(accuracy_score(test_labels, pred_labels_non_deskewed)))
  print("Accuracy - dane surowe: " + str(accuracy_score(test_labels, pred_labels_raw_deskewed)))
  print("Accuracy - dane zmieszane: " + str(accuracy_score(test_labels, pred_labels_shuffle)))

# Preprocessing



Surowe dane

In [ ]:
train_raw = train_images.reshape(len(train_images), 28 * 28)
test_raw = test_images.reshape(len(test_images), 28 * 28)

Dane poddane wyrównaniu

In [ ]:
train_deskewed = np.float32([deskew(im) for im in train_raw])
test_deskewed = np.float32([deskew(im) for im in test_raw])

train_deskewed = np.asarray(train_deskewed).reshape(-1,28*28)
test_deskewed = np.asarray(test_deskewed).reshape(-1,28*28)

In [ ]:
hogdata_train = np.float32([HOG(im) for im in train_images]).reshape(-1,81)
hogdata_test = np.float32([HOG(im) for im in test_images]).reshape(-1,81)

hogdata_train_deskewed = np.float32([HOG(deskew(im)) for im in train_images]).reshape(-1,81)
hogdata_test_deskewed = np.float32([HOG(deskew(im)) for im in test_images]).reshape(-1,81)

In [ ]:
# Zmieszane dane surowe
train_shuffle = train_images.copy()
train_shuffle = np.float32([deskew(im) for im in train_shuffle])
np.float32([np.random.shuffle(im) for im in train_shuffle])
train_shuffle = np.asarray(train_shuffle).reshape(-1,28*28)

test_shuffle = test_images.copy()
test_shuffle = np.float32([deskew(im) for im in test_shuffle])
np.float32([np.random.shuffle(im) for im in test_shuffle])
test_shuffle = np.asarray(test_shuffle).reshape(-1,28*28)

Dane dla walidacji krzyżowej

In [ ]:
hogdata_train_short = hogdata_train[:600]
hogdata_train_deskewed_short = hogdata_train_deskewed[:600]

# Surowe dane 
rawdata_train_deskewed_short = train_deskewed[:600]

# Zmieszane dane surowe 
train_shuffle_short = train_shuffle[:600]

train_labels_short = train_labels[:600]

# Model SVM

## Walidacja krzyżowa

Dane wyrównane

In [ ]:
model_deskewed = SVMGridSearch(hogdata_train_deskewed_short, train_labels_short)

Best hiperparameters:  {'C': 0.7353265306122448, 'kernel': 'linear'}


Dane niewyrównane

In [ ]:
model_non_deskewed = SVMGridSearch(hogdata_train_short, train_labels_short)

Best hiperparameters:  {'C': 0.8577142857142857, 'kernel': 'linear'}


Dane surowe

In [ ]:
model_raw_deskewed = SVMGridSearch(rawdata_train_deskewed_short, train_labels_short)

Best hiperparameters:  {'C': 0.001, 'kernel': 'linear'}


Zmieszane dane surowe

In [ ]:
model_shuffle = SVMGridSearch(train_shuffle_short, train_labels_short)

In [ ]:
## Trenowanie modelu

In [ ]:
%store model_deskewed
%store model_non_deskewed
%store model_raw_deskewed
%store model_shuffle

Stored 'model_deskewed' (SVC)
Stored 'model_non_deskewed' (SVC)
Stored 'model_raw_deskewed' (SVC)


In [ ]:
%store -r model_deskewed
%store -r model_non_deskewed
%store -r model_raw_deskewed
%store -r model_shuffle

Klasyfikacja

In [ ]:
model_deskewed.fit(hogdata_train_deskewed, train_labels)

pred_labels_deskewed = model_deskewed.predict(hogdata_test_deskewed)

In [ ]:
model_non_deskewed.fit(hogdata_train, train_labels)

pred_labels_non_deskewed = model_non_deskewed.predict(hogdata_test)

In [ ]:
idx = np.random.choice(np.arange(len(train_deskewed)), 6000, replace=False)
train_deskewed_random = train_deskewed[idx]
train_labels_random = train_labels[idx]

model_raw_deskewed.fit(train_deskewed_random,train_labels_random)
pred_labels_raw_deskewed = model_raw_deskewed.predict(test_deskewed)

In [ ]:
idx = np.random.choice(np.arange(len(train_shuffle)), 6000, replace=False)
train_shuffle_random = train_shuffle[idx]
train_labels_random = train_labels[idx]

model_shuffle.fit(train_shuffle_random,train_labels_random)
pred_labels_shuffle = model_shuffle.predict(test_shuffle)

Accuracy

In [ ]:
getAccuracy(test_labels, pred_labels_deskewed,pred_labels_non_deskewed,pred_labels_raw_deskewed,pred_labels_shuffle)

Accuracy - dane wyrównane: 0.9746
Accuracy - dane niewyrównane: 0.9651
Accuracy - dane surowe: 0.1135
Accuracy - dane zmieszane: 


Macierze pomyłek

In [ ]:
getConfMatrix(test_labels, pred_labels_deskewed,pred_labels_non_deskewed,pred_labels_raw_deskewed,pred_labels_shuffle)

Macierz pomyłek - dane wyrównane:
[[ 966    1    2    0    1    2    4    1    0    3]
 [   0 1125    1    2    0    0    2    4    1    0]
 [   0    0 1006    5    0    0    0   18    3    0]
 [   0    1    9  987    0    4    0    7    2    0]
 [   2    0    2    0  966    1    7    0    0    4]
 [   1    0    2    5    1  872    4    2    3    2]
 [   7    2    1    0    6    3  936    0    3    0]
 [   4    2   24    9    1    0    0  983    2    3]
 [   0    0    3   10    7    5    0    3  934   12]
 [   3    0    1    1    7    4    2    8   12  971]]
Macierz pomyłek - dane niewyrównane:
[[ 955    2    3    0    2    5    9    2    0    2]
 [   0 1124    3    0    0    0    1    5    2    0]
 [   3    1 1000    4    0    0    1   20    3    0]
 [   0    0    9  981    0    5    0   10    4    1]
 [   3    0    2    0  961    1    5    0    4    6]
 [   4    2    0   10    1  852    1    3   17    2]
 [  14    3    0    0    4    6  929    0    2    0]
 [   3    5   40    8    0 

Raporty

In [ ]:
getReports(test_labels, pred_labels_deskewed,pred_labels_non_deskewed,pred_labels_raw_deskewed,pred_labels_shuffle)

Raport - dane wyrównane:
              precision    recall  f1-score   support

           0       0.98      0.99      0.98       980
           1       0.99      0.99      0.99      1135
           2       0.96      0.97      0.97      1032
           3       0.97      0.98      0.97      1010
           4       0.98      0.98      0.98       982
           5       0.98      0.98      0.98       892
           6       0.98      0.98      0.98       958
           7       0.96      0.96      0.96      1028
           8       0.97      0.96      0.97       974
           9       0.98      0.96      0.97      1009

    accuracy                           0.97     10000
   macro avg       0.97      0.97      0.97     10000
weighted avg       0.97      0.97      0.97     10000

Raport - dane niewyrównane:
              precision    recall  f1-score   support

           0       0.97      0.97      0.97       980
           1       0.99      0.99      0.99      1135
           2       0.94  

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Wnioski

**TODO**


Użycie surowych danych spowodowało drastyczny wzrost czasu potrzebnego do trenowania modeli. Model który używa surowych danych był najmniej skutecznym (11% co jest bardzo bliskie do zgadywania), oraz trenowanie modelu z użyciem 6000 próbek trwało na 5 36 sekund minut dłużej niż modele które były trenowane na 60 000 próbkach przetworzonych danych. 


# Model RandomForest

## Walidacja krzyżowa

Dane wyrównane

In [ ]:
model_deskewed = RandomForestGridSearch(hogdata_train_deskewed_short, train_labels_short)

Fitting 3 folds for each of 500 candidates, totalling 1500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    6.5s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:   24.0s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:   54.4s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 1246 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 1500 out of 1500 | elapsed:  3.1min finished


Best hiperparameters:  {'max_depth': 15, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 70}


Dane niewyrównane

In [ ]:
model_non_deskewed = RandomForestGridSearch(hogdata_train_short, train_labels_short)

Fitting 3 folds for each of 500 candidates, totalling 1500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  88 tasks      | elapsed:   10.2s
[Parallel(n_jobs=-1)]: Done 388 tasks      | elapsed:   44.9s
[Parallel(n_jobs=-1)]: Done 888 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 1500 out of 1500 | elapsed:  3.0min finished


Best hiperparameters:  {'max_depth': 15, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 100}


Dane surowe

In [ ]:
model_raw_deskewed = RandomForestGridSearch(rawdata_train_deskewed_short, train_labels_short)

Fitting 3 folds for each of 500 candidates, totalling 1500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  88 tasks      | elapsed:   10.6s
[Parallel(n_jobs=-1)]: Done 388 tasks      | elapsed:   47.1s
[Parallel(n_jobs=-1)]: Done 888 tasks      | elapsed:  1.8min


Best hiperparameters:  {'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 20}


[Parallel(n_jobs=-1)]: Done 1500 out of 1500 | elapsed:  3.0min finished


Zmieszane dane surowe

In [ ]:
model_shuffle = RandomForestGridSearch(train_shuffle_short, train_labels_short)

## Trenowanie modelu

Klasyfikacja

In [ ]:
model_deskewed.fit(hogdata_train_deskewed, train_labels)
model_non_deskewed.fit(hogdata_train, train_labels)
model_raw_deskewed.fit(train_deskewed, train_labels)
model_shuffle.fit(train_shuffle, train_labels)

pred_labels_deskewed = model_deskewed.predict(hogdata_test_deskewed)
pred_labels_non_deskewed = model_non_deskewed.predict(hogdata_test)
pred_labels_raw_deskewed = model_raw_deskewed.predict(test_deskewed)
pred_labels_shuffle = model_shuffle.predict(test_shuffle)

Accuracy

In [ ]:
getAccuracy(test_labels, pred_labels_deskewed,pred_labels_non_deskewed,pred_labels_raw_deskewed,pred_labels_shuffle)

Accuracy - dane wyrównane: 0.9691
Accuracy - dane niewyrównane: 0.9586
Accuracy - dane surowe: 0.1135
Accuracy - dane zmieszane: 


Macierz pomyłek

In [ ]:
getConfMatrix(test_labels, pred_labels_deskewed,pred_labels_non_deskewed,pred_labels_raw_deskewed,pred_labels_shuffle)

Macierz pomyłek - dane wyrównane:
[[ 961    0    2    0    0    4    3    2    3    5]
 [   0 1125    2    0    0    0    1    4    3    0]
 [   1    1 1004    2    1    0    2   15    4    2]
 [   0    2   10  980    0    5    0    9    2    2]
 [   2    1    0    0  953    2    8    0    4   12]
 [   4    0    0    3    0  875    2    1    4    3]
 [   6    3    0    0    9    6  930    0    3    1]
 [   3    1   26    9    0    0    0  977    5    7]
 [   0    0    2    7    7    6    4    4  919   25]
 [   6    0    1    0    3    7    3    2   20  967]]
Macierz pomyłek - dane niewyrównane:
[[ 955    1    2    0    5    8    5    1    0    3]
 [   0 1123    4    0    1    1    1    3    2    0]
 [   3    1  991    4    2    0    3   19    6    3]
 [   0    0   13  978    0    4    0   10    5    0]
 [   5    1    0    0  944    2   13    0    1   16]
 [   3    3    1    7    0  858    1    3   11    5]
 [  13    3    1    0    6   10  922    0    2    1]
 [   2    1   35   17    0 

Raporty

In [ ]:
getReports(test_labels, pred_labels_deskewed,pred_labels_non_deskewed,pred_labels_raw_deskewed,pred_labels_shuffle)

Raport - dane wyrównane:
              precision    recall  f1-score   support

           0       0.98      0.98      0.98       980
           1       0.99      0.99      0.99      1135
           2       0.96      0.97      0.97      1032
           3       0.98      0.97      0.97      1010
           4       0.98      0.97      0.97       982
           5       0.97      0.98      0.97       892
           6       0.98      0.97      0.97       958
           7       0.96      0.95      0.96      1028
           8       0.95      0.94      0.95       974
           9       0.94      0.96      0.95      1009

    accuracy                           0.97     10000
   macro avg       0.97      0.97      0.97     10000
weighted avg       0.97      0.97      0.97     10000

Raport - dane niewyrównane:
              precision    recall  f1-score   support

           0       0.96      0.97      0.97       980
           1       0.99      0.99      0.99      1135
           2       0.94  

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Wnioski

**TODO**
Stosowanie danych surowych do trenowania modelu powoduje nadmiarowe zużycie pamięci, przez co czas trenowania i predykcji jest wielokrotnie większy w porównaniu z użyciem danych poprzednie 



# Model Neural Network

## Stworzenie modelu

In [ ]:
network_deskewed = create_model()
network_non_deskewed = create_model()
network_raw_deskewed = create_model()
network_shuffle = create_model()

In [ ]:
nn_train_deskewed = np.array(hogdata_train_deskewed).reshape((60000, 81))
nn_train_deskewed = nn_train_deskewed.astype('float32') / 255

nn_test_deskewed = np.array(hogdata_test_deskewed).reshape((10000, 81))
nn_test_deskewed = nn_test_deskewed.astype('float32') / 255

nn_train_non_deskewed = np.array(hogdata_train).reshape((60000, 81))
nn_train_non_deskewed = nn_train_non_deskewed.astype('float32') / 255

nn_test_non_deskewed = np.array(hogdata_test).reshape((10000, 81))
nn_test_non_deskewed = nn_test_non_deskewed.astype('float32') / 255

nn_train_raw_deskewed = np.array(train_deskewed).reshape((60000, 28*28))
nn_train_raw_deskewed = nn_train_raw_deskewed.astype('float32') / 255

nn_test_raw_deskewed = np.array(test_deskewed).reshape((10000, 28*28))
nn_test_raw_deskewed = nn_test_raw_deskewed.astype('float32') / 255

nn_train_shuffle = np.array(train_shuffle).reshape((60000, 28*28))
nn_train_shuffle = nn_train_shuffle.astype('float32') / 255

nn_test_shuffle = np.array(test_shuffle).reshape((10000, 28*28))
nn_test_shuffle = nn_test_shuffle.astype('float32') / 255

In [ ]:
encoded_train_labels = to_categorical(train_labels)
encoded_test_labels = to_categorical(test_labels)

## Walidacja krzyżowa

Dane wyrównane

In [ ]:
model_deskewed = NeuralNetworkGridSrearch(nn_train_deskewed, encoded_train_labels)

/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Best hiperparameters:  {'batch_size': 30, 'epochs': 15}


Dane niewyrównane

In [ ]:
model_non_deskewed = NeuralNetworkGridSrearch(nn_train_non_deskewed, encoded_train_labels)

Best hiperparameters:  {'batch_size': 30, 'epochs': 15}


Dane surowe

In [ ]:
model_raw_deskewed = NeuralNetworkGridSrearch(nn_train_raw_deskewed, encoded_train_labels)

Best hiperparameters:  {'batch_size': 30, 'epochs': 5}


Zmieszane dane surowe

In [ ]:
model_shuffle = NeuralNetworkGridSrearch(nn_train_shuffle, encoded_train_labels)

## Trenowanie modelu

Dane wyrównane

In [ ]:
network_deskewed.fit(nn_train_deskewed, encoded_train_labels, epochs=model_deskewed.get('epochs'), batch_size=model_deskewed.get('batch_size'))

Epoch 1/15
2000/2000 [==============================] - 3s 1ms/step - loss: 2.0606 - accuracy: 0.3341
Epoch 2/15
2000/2000 [==============================] - 3s 1ms/step - loss: 1.0054 - accuracy: 0.7758
Epoch 3/15
2000/2000 [==============================] - 3s 1ms/step - loss: 0.5596 - accuracy: 0.8573
Epoch 4/15
2000/2000 [==============================] - 3s 1ms/step - loss: 0.3998 - accuracy: 0.8887
Epoch 5/15
2000/2000 [==============================] - 3s 1ms/step - loss: 0.3159 - accuracy: 0.9114
Epoch 6/15
2000/2000 [==============================] - 3s 1ms/step - loss: 0.2658 - accuracy: 0.9232
Epoch 7/15
2000/2000 [==============================] - 3s 1ms/step - loss: 0.2320 - accuracy: 0.9304
Epoch 8/15
2000/2000 [==============================] - 3s 1ms/step - loss: 0.2052 - accuracy: 0.9393
Epoch 9/15
2000/2000 [==============================] - 3s 1ms/step - loss: 0.1903 - accuracy: 0.9440
Epoch 10/15
2000/2000 [==============================] - 3s 1ms/step - loss: 0.180

Dane niewyrównane

In [ ]:
network_non_deskewed.fit(nn_train_non_deskewed, encoded_train_labels, epochs=model_non_deskewed.get('epochs'), batch_size=model_non_deskewed.get('batch_size'))

Epoch 1/15
2000/2000 [==============================] - 3s 1ms/step - loss: 2.1051 - accuracy: 0.3088
Epoch 2/15
2000/2000 [==============================] - 3s 1ms/step - loss: 1.1345 - accuracy: 0.7191
Epoch 3/15
2000/2000 [==============================] - 3s 1ms/step - loss: 0.6986 - accuracy: 0.8093
Epoch 4/15
2000/2000 [==============================] - 3s 1ms/step - loss: 0.5242 - accuracy: 0.8497
Epoch 5/15
2000/2000 [==============================] - 3s 1ms/step - loss: 0.4246 - accuracy: 0.8774
Epoch 6/15
2000/2000 [==============================] - 3s 1ms/step - loss: 0.3603 - accuracy: 0.8959
Epoch 7/15
2000/2000 [==============================] - 3s 1ms/step - loss: 0.3232 - accuracy: 0.9038
Epoch 8/15
2000/2000 [==============================] - 3s 1ms/step - loss: 0.2956 - accuracy: 0.9093
Epoch 9/15
2000/2000 [==============================] - 3s 1ms/step - loss: 0.2726 - accuracy: 0.9167
Epoch 10/15
2000/2000 [==============================] - 3s 1ms/step - loss: 0.255

Dane surowe

In [ ]:
network_raw_deskewed.fit(nn_train_raw_deskewed, encoded_train_labels, epochs=model_raw_deskewed.get('epochs'), batch_size=model_raw_deskewed.get('batch_size'))

Epoch 1/5
2000/2000 [==============================] - 3s 1ms/step - loss: 0.1440 - accuracy: 0.9549
Epoch 2/5
2000/2000 [==============================] - 3s 1ms/step - loss: 0.1409 - accuracy: 0.9558
Epoch 3/5
2000/2000 [==============================] - 3s 1ms/step - loss: 0.1380 - accuracy: 0.9565
Epoch 4/5
2000/2000 [==============================] - 3s 1ms/step - loss: 0.1353 - accuracy: 0.9577
Epoch 5/5
2000/2000 [==============================] - 3s 1ms/step - loss: 0.1329 - accuracy: 0.9583


Zmieszane dane surowe

In [ ]:
network_shuffle.fit(nn_train_shuffle,encoded_train_labels, epochs=model_shuffle.get('epochs'), batch_size=model_shuffle.get('batch_size'))

In [ ]:
pred_probabilities_deskewed = network_deskewed.predict(nn_test_deskewed)
pred_probabilities_non_deskewed = network_non_deskewed.predict(nn_test_non_deskewed)
pred_probabilities_raw_deskewed = network_raw_deskewed.predict(nn_test_raw_deskewed)
pred_probabilities_shuffle = network_shuffle.predict(nn_test_shuffle)

pred_labels_deskewed = np.argmax(pred_probabilities_deskewed,-1)
pred_labels_non_deskewed = np.argmax(pred_probabilities_non_deskewed,-1)
pred_labels_raw_deskewed = np.argmax(pred_probabilities_raw_deskewed,-1)
pred_labels_shuffle = np.argmax(pred_probabilities_shuffle,-1)

Accuracy

In [ ]:
getAccuracy(test_labels, pred_labels_deskewed,pred_labels_non_deskewed,pred_labels_raw_deskewed,pred_labels_shuffle)

Accuracy - dane wyrównane: 0.9597
Accuracy - dane niewyrównane: 0.9425
Accuracy - dane surowe: 0.1135
Accuracy - dane zmieszane: 


Macierz pomyłek

In [ ]:
getConfMatrix(test_labels, pred_labels_deskewed,pred_labels_non_deskewed,pred_labels_raw_deskewed,pred_labels_shuffle)

Macierz pomyłek - dane wyrównane:
[[ 961    0    3    0    1    3    5    1    2    4]
 [   3 1117    2    1    3    0    3    4    1    1]
 [   0    2  990   14    0    0    1   18    7    0]
 [   0    0    7  986    0    8    0    7    2    0]
 [   2    1    3    0  953    1   10    1    2    9]
 [   5    0    1    5    0  873    2    1    3    2]
 [   7    3    1    0    6    4  933    0    4    0]
 [   3    4   41   32    0    1    0  934    3   10]
 [   1    0    3   17   10    5    1    3  920   14]
 [   9    1    4    4    7   16    4   13   21  930]]
Macierz pomyłek - dane niewyrównane:
[[ 933    3    6    0    6    7   15    3    5    2]
 [   0 1123    2    1    5    0    2    0    2    0]
 [   2    4  972    8    1    1    0   31   11    2]
 [   0    0   16  968    0    9    0   10    6    1]
 [   3    3    1    0  951    1   13    0    3    7]
 [   3    1    2    8    2  825    2    2   39    8]
 [  12    3    1    0   17    6  912    0    6    1]
 [   4    5   60   20    1 

Raporty

In [ ]:
getReports(test_labels, pred_labels_deskewed,pred_labels_non_deskewed,pred_labels_raw_deskewed,pred_labels_shuffle)

Raport - dane wyrównane:
              precision    recall  f1-score   support

           0       0.97      0.98      0.98       980
           1       0.99      0.98      0.99      1135
           2       0.94      0.96      0.95      1032
           3       0.93      0.98      0.95      1010
           4       0.97      0.97      0.97       982
           5       0.96      0.98      0.97       892
           6       0.97      0.97      0.97       958
           7       0.95      0.91      0.93      1028
           8       0.95      0.94      0.95       974
           9       0.96      0.92      0.94      1009

    accuracy                           0.96     10000
   macro avg       0.96      0.96      0.96     10000
weighted avg       0.96      0.96      0.96     10000

Raport - dane niewyrównane:
              precision    recall  f1-score   support

           0       0.96      0.95      0.95       980
           1       0.98      0.99      0.99      1135
           2       0.91  

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Wnioski

**TODO**

# Podsumowanie

**TODO**